In [1]:
import wisps
import splat
import wisps.simulations as wispsim
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
from astropy.coordinates import SkyCoord
from tqdm import tqdm

%matplotlib inline

In [2]:
lf=wisps.LUMINOSITY_FUCTION
lfdes=wisps.DES_LUMINOSITY_FUCTION
maglimits=wisps.MAG_LIMITS

In [3]:
from wisps.simulations import BayesianPointing

In [4]:
obs=pd.read_csv(wisps.OUTPUT_FILES+'//observation_log_with_limit.csv')

In [5]:
obs.columns

Index(['Unnamed: 0', 'pointing', 'bright_F110W', 'faint_F110W', 'bright_F140W',
       'faint_F140W', 'bright_F160W', 'faint_F160W', 'ra (deg)', 'dec(deg)',
       'l (deg)', 'b (deg)', 'exposure (s)', 'observation date (ut)'],
      dtype='object')

In [6]:
def make_pointing(ra, dec, survey, name):
    coord=SkyCoord(ra=ra*u.deg,dec=dec*u.deg )
    return BayesianPointing(coord=coord, survey=survey, name=name)

In [7]:
def get_survey(pointing):
    if pointing.startswith('par'):
        return 'wisps'
    else:
        return 'hst3d'

In [8]:
ras=obs['ra (deg)']
decs=obs['dec(deg)']
surveys=obs.pointing.apply(get_survey)

In [9]:
pnts=[make_pointing(ra, dec, survey, name) for ra, dec, survey, name in zip(ras, decs, surveys, obs.pointing.values)]

In [10]:
from numba import cuda
import numba

In [11]:
#@numba.vectorize("float64(float64, float64)", target='cuda')
#@cuda.jit
#def add_numbers(x, y):
#    return x+y

In [12]:
#import numpy as np

In [13]:
#%%prun 
#x=np.random.uniform(100)
#add_numbers(x, x)

In [14]:
#

In [15]:
for pnt in tqdm(pnts):
    pnt.mag_limits=maglimits[pnt.survey]
    #print( pnt.survey)
    pnt.compute_distance_limits()
    pnt.computer_volume()
    #pnt.create_sample( nsample=1000)

100%|██████████| 533/533 [00:22<00:00, 23.20it/s]


In [16]:
import pickle
#save the random forest
output_file=wisps.OUTPUT_FILES+'/bayesian_observed_pointings.pkl'
with open(output_file, 'wb') as file:
    pickle.dump(pnts,file)
